In [3]:
!pip install numpy scipy pandas mne scikit-learn tensorflow matplotlib


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install --upgrade pip

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1
Note: you may need to restart the kernel to use updated packages.


In [19]:
import numpy as np
import os
import pickle

# configutation
DATA_PATH = r"C:\Users\Sameera\.cache\kagglehub\datasets\manh123df\deap-dataset\versions\1\deap-dataset\data_preprocessed_python"

VALENCE_THRESHOLD = 5.0 # High vs low valance
TIME_WINDOW = 60 # Duration per trail in seconds
SAMPLING_RATE = 128 # Hz

# Load all subjects' data
subject_files = sorted([f for f in os.listdir(DATA_PATH) if f.endswith('.dat')])

all_data = []
all_labels = []

for file_name in subject_files:
    file_path = os.path.join(DATA_PATH, file_name)
    print(f"Loading: {file_name}")

    # Each .dat file is a pickled Python dictionary
    with open(file_path, 'rb') as f:
        subject_data = pickle.load(f, encoding='latin1')

    # Extract EEG data (first 32 channles)
    eeg_data = subject_data['data'][:, 0:32, :] # shape: (40, 32, 8064)

    # Extract labels (valence, arousal, dominance, liking)
    labels = subject_data['labels']

    # Binary valence labels (0 = Low, 1 = High)
    valence_labels = (labels[:, 0] >= VALENCE_THRESHOLD).astype(int)

    all_data.append(eeg_data)
    all_labels.append(valence_labels)

# combine all subjects
X = np.concatenate(all_data, axis = 0) # shape : (32 subjects * 40 trials, 32, 8064)
Y = np.concatenate(all_labels, axis = 0) # shape : (32 subjects * 40 trails)

print("\nData Loaded Successfully!")
print(f"Total EEG trials (X) : {X.shape}")
print(f"Total Labels (Y) : {Y.shape}")

Loading: s01.dat
Loading: s02.dat
Loading: s03.dat
Loading: s04.dat
Loading: s05.dat
Loading: s06.dat
Loading: s07.dat
Loading: s08.dat
Loading: s09.dat
Loading: s10.dat
Loading: s11.dat
Loading: s12.dat
Loading: s13.dat
Loading: s14.dat
Loading: s15.dat
Loading: s16.dat
Loading: s17.dat
Loading: s18.dat
Loading: s19.dat
Loading: s20.dat
Loading: s21.dat
Loading: s22.dat
Loading: s23.dat
Loading: s24.dat
Loading: s25.dat
Loading: s26.dat
Loading: s27.dat
Loading: s28.dat
Loading: s29.dat
Loading: s30.dat
Loading: s31.dat
Loading: s32.dat

Data Loaded Successfully!
Total EEG trials (X) : (1280, 32, 8064)
Total Labels (Y) : (1280,)
